# Nutrient optimization

## todo

* parallel GA with deap
    * DONE: that was too easy!
* API
* set initial seed population with clusters
* GA: remove nutrients that are uncontrained when solving (eg, 1e10)
    * DONE: this doesn't speed up the function
* random seed doesn't work

## Import custom helper functions

* def load_data():
* def do_clust(N,lim,req,nut):
* def evaluate(individual, nut,limt,reqd):
* def makeclusters(nclust,limt,reqd,nutrients ):
* def InitPopulation( pcls, ind_init,nfood, nclust, nseed,clust):
* def generate_ssdum(random, args):

In [1]:
import sys
print(sys.path)
from lib.libraries import *

['/home/pedwards/documents/SpartanSupper/diet_ga', '/home/pedwards/.conda/envs/spartansupper/lib/python37.zip', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/lib-dynload', '', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/site-packages', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/site-packages/IPython/extensions', '/home/pedwards/.ipython']


## Import external libraries

Most are standard, but we want the glpk solver for cvxopt, which requires the following,
```
$ sudo apt-get install libglpk-dev
$ sudo CVXOPT_BUILD_GLPK=1 pip install cvxopt
```

In [2]:
from os import path

import pickle
import pandas
import numpy
from deap import base, creator, tools, algorithms
from sklearn.preprocessing import normalize
from cvxopt import matrix, solvers # an alternative linear programming library
solvers.options['show_progress'] = False
solvers.options['glpk'] = {'msg_lev' : 'GLP_MSG_OFF'} #mute all output from glpk

from sklearn.cluster import KMeans
import random
from time import time
import multiprocessing

pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)

## Load food & nutrients from database

In [3]:
(nutrients,reqd,limt,food_desc,nutrient_desc)=load_data()
print( '[*] Loaded %d foods from database' % nutrients.shape[0] )

[*] Loaded 4482 foods from database


## Constants

In [4]:
N_FOODS=6
Nseed=500
NT_DIM=nutrients.shape[0]
Nclust=15

## Cluster

Observation: the optimization converges faster (and to lower minima) if provided a "seed" population with random baskets of *diverse* foods

Technique: I use a kmeans to find clusters of food types then sample from them (with a multinomial dist)

In [5]:
cluster_food_count=0

if path.exists('clust.pkl'):
    clust=pickle.load( open( "clust.pkl", "rb" ) )
    print( '[*] Found pickle file with %d clusters and %d foods' % (clust.max()+1,len(clust)) )
    Nclust=clust.max()+1
    cluster_food_count=len(clust)

if cluster_food_count != nutrients.shape[0] :
    print( '[*] Rebuilding clusters...' )
    clust=makeclusters(Nclust,limt,reqd,nutrients )
    pickle.dump( clust, open( "clust.pkl", "wb" ) )

[*] Found pickle file with 15 clusters and 4482 foods


## Genetic algorithm

This is the outermost optimization layer (the inner optim is in `evaluate()`)


Todo: play with early stopping. no need to keep going after improvement has ceased or slowed.

In [6]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin) # an individual comprises a list (of food IDs)

toolbox = base.Toolbox()
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)
# Attribute generator 
toolbox.register("attr_foodid", random.randrange, NT_DIM)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_foodid, N_FOODS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=NT_DIM, indpb=0.1)
#toolbox.register("select", tools.selBest, k=3)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("evaluate", evaluate, nut=nutrients,limt=limt,reqd=reqd)

# used to make a seed population (only) ; per: https://deap.readthedocs.io/en/master/tutorials/basic/part1.html?highlight=seeding#seeding-a-population
toolbox.register("population_guess", InitPopulation, list, creator.Individual, N_FOODS,Nclust,Nseed,limt,reqd,nutrients )

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("min", numpy.min)
stats.register("median", numpy.median)
stats.register("max", numpy.max)

In [7]:
%%time

pop = toolbox.population(n=300) # totally random initial population
#pop = toolbox.population_guess()
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, 
                                   stats=stats, verbose=True)

gen	nevals	min    	median	max  
0  	300   	1355.28	9e+09 	9e+09
1  	188   	1341.12	9e+09 	9e+09
2  	174   	1290.75	2822.88	9e+09
3  	202   	1250.75	1355.28	9e+09
4  	169   	1250.75	1341.12	9e+09
5  	181   	1250.75	1290.75	9e+09
6  	190   	1250.75	1250.75	9e+09
7  	183   	1250.75	1250.75	9e+09
8  	180   	1160.89	1250.75	9e+09
9  	182   	1160.89	1250.75	9e+09
10 	159   	983.393	1250.75	9e+09
11 	178   	947.921	1160.89	9e+09
12 	172   	878.11 	1160.89	9e+09
13 	184   	878.11 	953.701	9e+09
14 	182   	878.11 	878.11 	9e+09
15 	189   	798.57 	878.11 	9e+09
16 	193   	798.57 	878.11 	9e+09
17 	166   	798.57 	878.11 	9e+09
18 	177   	722.326	798.57 	9e+09
19 	178   	722.326	798.57 	9e+09
20 	178   	722.326	798.57 	9e+09
21 	189   	669.606	722.326	9e+09
22 	182   	669.606	722.326	9e+09
23 	174   	669.606	707.354	9e+09
24 	170   	669.606	669.606	9e+09
25 	181   	669.606	669.606	9e+09
26 	188   	669.606	669.606	9e+09
27 	188   	669.606	669.606	9e+09
28 	169   	614.994	669.606	9e+09
29 	185   	61

In [8]:
best=tools.selBest(pop, k=1)
best=best[0]
evaluate(best, nut=nutrients,limt=limt,reqd=reqd)
nt=nutrients.iloc[best,:]
c = matrix(numpy.repeat(1.0,nt.shape[0]))
np_G= numpy.concatenate(
                        (   nt.transpose().values, 
                            nt.transpose().multiply(-1.0).values,
                            numpy.diag(numpy.repeat(-1,nt.shape[0])) 
                        )
                       ).astype(numpy.double) 
G = matrix( np_G ) 
h = matrix( numpy.concatenate( (
                limt.values, 
                reqd.multiply(-1.0).values, 
                numpy.repeat(0.0,nt.shape[0])
            ) ).astype(numpy.double) )    
o = solvers.lp(c, G, h, solver='glpk')
food_amounts=numpy.array(o['x'])[:,0]


## Print the best "diet"

This is the food and corresponding amount to eat (in grams, sorry bud!). The idea is if you eat all this in a day you have the nutrients you need for the day.

Todo: this would be better if split up into recipes. 

In [9]:
final_foods= [ best[i] for i in range(len(food_amounts)) if food_amounts[i]>1e-6 ] # select those with non-trivial amounts
final_food_amounts= food_amounts[ food_amounts>1e-6 ]

nt=nutrients.iloc[final_foods,:]
df1= nt.join(food_desc).loc[:,['long_desc']] #food_desc.iloc[final_foods]
df2=pandas.DataFrame(final_food_amounts*100,index=df1.index,columns=["amount"])
df3=pandas.DataFrame(nt.loc[:,208].values * df2.loc[:,'amount'].values/100 ,columns=['calories'], index=df2.index)
diet_table=df1.join(df2).join(df3)

In [15]:
diet_table.to_json(orient='split')

'{"columns":["long_desc","amount","calories"],"index":[13324,11012,15141,11623,11939],"data":[["Beef, variety meats and by-products, kidneys, cooked, simmered",20.6402737786,32.6116325702],["Asparagus, cooked, boiled, drained",381.4323117575,83.9151085866],["Crustaceans, crab, blue, canned",340.3262495319,282.4707871115],["Kale, scotch, cooked, boiled, drained, without salt",297.2144665413,83.2200506316],["Mushrooms, portabella, exposed to ultraviolet light, grilled",210.964445641,61.1796892359]]}'

In [11]:
nutrient_totals=pandas.DataFrame( ( 
                    numpy.dot( numpy.transpose(final_food_amounts), nt.values),
                    reqd,
                    limt
                  ), index=['Total','Amount required','Amount limit'], columns=nt.columns).transpose()

In [12]:
A=nt.join(df1).set_index('long_desc').transpose() * final_food_amounts
nutrient_report=A.join(nutrient_desc).join(nutrient_totals).set_index('name')
nutrient_report

,"Beef, variety meats and by-products, kidneys, cooked, simmered","Asparagus, cooked, boiled, drained","Crustaceans, crab, blue, canned","Kale, scotch, cooked, boiled, drained, without salt","Mushrooms, portabella, exposed to ultraviolet light, grilled",Total,Amount required,Amount limit
name,,,,,,,,
Protein,5.628603,9.154376,60.850327,5.647075,6.919633,88.200018,53.459999,1.000000e+10
Total lipid (fat),0.959773,0.839151,2.518414,1.218579,1.223594,6.759511,0.000000,1.000000e+10
"Carbohydrate, by difference",0.000000,15.676868,0.000000,16.733175,9.366821,41.776865,0.000000,1.000000e+10
Ash,0.270388,2.403023,6.670394,2.585766,2.215127,14.144698,0.000000,1.000000e+10
Energy,32.611633,83.915108,282.470764,83.220047,61.179688,543.397268,0.000000,1.000000e+10
Starch,0.000000,0.000000,0.000000,0.000000,0.907147,0.907147,0.000000,1.000000e+10
Sucrose,0.000000,0.305146,0.000000,0.000000,0.000000,0.305146,0.000000,1.000000e+10
Glucose (dextrose),0.000000,1.602016,0.000000,0.000000,4.767797,6.369812,0.000000,1.000000e+10
Fructose,0.000000,3.013315,0.000000,0.000000,0.000000,3.013315,0.000000,1.000000e+10
